In [30]:
from pandas import DataFrame as df
from typing import Optional, Union
from datetime import date
from pathlib import Path
import pandas as pd
from os import path
import numpy as np
import datetime
import typing
import time
import os

pd.options.display.float_format = "{:,.2f}".format
pd.options.mode.copy_on_write = True

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
test_dt = date(2024, 1, 17)
test_dt.replace(day=15)
test_dt

datetime.date(2024, 1, 17)

In [ ]:
"""
NOTE -- For Winterfell:
Segment Start Date: The 2nd BD occuring after the 13th calendar day of the month
Segment Expiry Date:  The 1st BD occuring after the 13th calendar day of the month
"""


# Function to Calc HedgeDate by adjusting the ExpiryDt back by Mat(Yrs) date by years and setting day to 13 
def adjust_date(row): 
    new_date = row['ExpiryDt'] + pd.DateOffset(years=-1*row['Mat(Yrs)'])
    new_date = new_date.replace(day=13)
    # Now Adjust forward by 2BD

    return new_date
    

In [97]:
iul_rpt_file = r'C:\Users\S0053071\Repos\Winterfell\IUL_Rpt_File\IUL_Report_20241022.txt'
idx_map_file = r'C:\Users\S0053071\Repos\Winterfell\IUL_Rpt_File\Index_Mapping.xlsx'

iul_inforce_file = r'C:\Users\S0053071\Repos\Winterfell\IUL_Rpt_File\IUL_Inforce_20241022.xlsx'

In [64]:
idx_map = pd.read_excel(idx_map_file)
idx_map

,Index_Orig,Bbg_Idx,Mat(Yrs),Part
0,*** S&P 500 - 1 YEAR TERM ***,SPX Index,1,1.00
1,*** EAFE ***,MXEA Index,1,1.00
2,*** RUSSELL ***,RTY Index,1,1.00
3,*** S&P 500 - 3 YEAR TERM ***,SPX Index,3,1.00
4,*** CH S&P 500 - 1 YEAR TERM ***,SPX Index,1,1.00
5,*** CH EAFE ***,MXEA Index,1,1.00
6,*** CH RUSSELL ***,RTY Index,1,1.00
7,*** CH S&P 500 - 3 YEAR TERM ***,SPX Index,3,1.00
8,***PLUS S&P 500 - 1 YEAR TERM***,SPX Index,1,1.00
9,*** PLUS EAFE ***,MXEA Index,1,1.00


In [ ]:
indices = []
tbl_data = []
filtered_lines = []
data_rows = []
last_idx = None
line_num = 0
updated_headers = False
headers = []

with open(iul_rpt_file, 'r') as file:

    for line in file:

        line_num = line_num + 1

        if ((line.split(',')[0].strip()).startswith('COMPANY'))==True:
            # print(f'Skipping line {line_num}, as it starts w/ Company!')
            continue
        
        comma_count = line.count(",")

        if comma_count == 2:
            # Grab the Index field that is applicable to the next section
            elem_3 = line.split(',')[2].strip()
            if elem_3.startswith('*'):
                # print(f'The 3rd element of line {line_num} is {elem_3}')
                # filtered_lines.append(elem_3)
                print(f'Setting {elem_3} to the last read index: {elem_3}')
                last_idx = elem_3
                indices.append(elem_3)
            else:
                continue
        elif comma_count < 12:
            continue
        else:
            # Grab and clean the data
            line_split = line.split(',')
            values = [val.strip() for val in line_split[:11]]

            # If true, We are dealing with a header row
            if values[2]=='FUND':
                if updated_headers:
                    # Continue, since we've already figured out what the headers are.  We just need to append new rows
                    continue
                else:
                    # Update the values in the header field to actual headers
                    values[0] = 'CompID'
                    values[1] = 'Series'
                    # Insert a header and place holder for storing the index value
                    values.insert(2, 'Index_Orig')
                    headers = values
            else:
                values.insert(2, last_idx)
                data_rows.append(values)
 

Setting *** S&P 500 - 1 YEAR TERM *** to the last read index: *** S&P 500 - 1 YEAR TERM ***
Setting *** EAFE *** to the last read index: *** EAFE ***
Setting *** RUSSELL *** to the last read index: *** RUSSELL ***
Setting *** S&P 500 - 3 YEAR TERM *** to the last read index: *** S&P 500 - 3 YEAR TERM ***
Setting *** S&P 500 - 1 YEAR TERM *** to the last read index: *** S&P 500 - 1 YEAR TERM ***
Setting *** EAFE *** to the last read index: *** EAFE ***
Setting *** RUSSELL *** to the last read index: *** RUSSELL ***
Setting *** S&P 500 - 1 YEAR TERM *** to the last read index: *** S&P 500 - 1 YEAR TERM ***
Setting *** EAFE *** to the last read index: *** EAFE ***
Setting *** RUSSELL *** to the last read index: *** RUSSELL ***
Setting *** S&P 500 - 3 YEAR TERM *** to the last read index: *** S&P 500 - 3 YEAR TERM ***
Setting *** CH S&P 500 - 1 YEAR TERM *** to the last read index: *** CH S&P 500 - 1 YEAR TERM ***
Setting *** CH EAFE *** to the last read index: *** CH EAFE ***
Setting *** 

In [ ]:
iul_rpt_df =  pd.DataFrame(data_rows, columns=headers)

iul_rpt_df['MATURITY'] = pd.to_datetime(iul_rpt_df['MATURITY']).dt.date
iul_rpt_df['MATURITY'] = iul_rpt_df['MATURITY'].replace('', np.nan)

filtered_df = iul_rpt_df[iul_rpt_df['MATURITY'].notna()]
filtered_df = filtered_df[filtered_df['CAPR']!='']

float_cols = ['CAPR', 'SEG_VALUE', 'SEG_PRIN_AMT', 'PROJ_CRED', 'LOAN', 'PW', 'MD']
for col in float_cols:
    filtered_df[col] = filtered_df[col].astype(float)

filtered_df = filtered_df[filtered_df['SEG_VALUE']!=0]
filtered_df['ExpiryDt'] = filtered_df['MATURITY']
filtered_df['Notional'] = filtered_df['SEG_VALUE']
filtered_df['Cap'] = filtered_df['CAPR'] / 100.0
filtered_df['Product'] = 'IUL'
filtered_df['Opt_Type'] = 'Call Spread'
filtered_df['Buffer'] = 1

# Merge w/ Helper Table to Map Index to Bbg_Idx, Mat(Yrs) and Part Rt
filtered_df = filtered_df.merge(idx_map, on=['Index_Orig'])

# SET THE VAL DATE!
val_dt = date(2024, 10, 22)
filtered_df['ValDt'] = val_dt

# Remove Expired Data by only keeping data for which the expiration is after the valuation date (no need to price the past)
filtered_df = filtered_df[filtered_df['ExpiryDt'] >= filtered_df['ValDt']]

# Calculate/Add the HedgeDate by calling function that adjusts expiry date backwards by Mat(Yrs)
# NOTE -- NEED TO MAKE SURE THIS IS A VALID BUSINESS DATE TO GET MKT PRICES!  
filtered_df['HedgeDate'] = filtered_df.apply(adjust_date, axis=1)

# NOTE:  Will drop HedgeDate because it needs to be calculating using Numerix in order to get the correct StartDate (2BD from the 13th),  Just using HedgeDate as proxy to remove data where HedgeDate > ValDt (aka, hasn't started yet)
# Remove Data for which the Effective/HedgeDate is after the Valuation Date (since the segment hasn't started yet, but is present due to data issues).  Accomplish by only including records for which HedgeDate <Effective/Start Dt> is B4 the Valuation Date!
filtered_df = filtered_df[filtered_df['HedgeDate'] <= filtered_df['ValDt']]


# NOTE:  Removed HedgeDate.  It can be calculated using Nx later, when Nx is needed for more things!
# ordered_cols_to_keep = ['ValDt', 'Product', 'Opt_Type', 'CompID', 'Series', 'FUND', 'Bbg_Idx', 'Notional', 'HedgeDate', 'ExpiryDt', 'Mat(Yrs)', 'Cap', 'Buffer','Part']
ordered_cols_to_keep = ['ValDt', 'Product', 'Opt_Type', 'CompID', 'Series', 'FUND', 'Bbg_Idx', 'Notional', 'ExpiryDt', 'Mat(Yrs)', 'Cap', 'Buffer','Part']
filtered_df = filtered_df[ordered_cols_to_keep]




# Write the results to file
filtered_df.to_excel(iul_inforce_file, index=False)


In [100]:
filtered_df['ExpiryDt'].unique()

array([datetime.date(2027, 6, 14), datetime.date(2025, 1, 14),
       datetime.date(2025, 2, 14), datetime.date(2025, 3, 14),
       datetime.date(2025, 4, 14), datetime.date(2025, 5, 14),
       datetime.date(2025, 6, 16), datetime.date(2025, 7, 14),
       datetime.date(2025, 8, 14), datetime.date(2025, 9, 15),
       datetime.date(2025, 10, 14), datetime.date(2024, 11, 14),
       datetime.date(2024, 12, 16), datetime.date(2025, 11, 14),
       datetime.date(2025, 12, 15), datetime.date(2026, 1, 14),
       datetime.date(2026, 2, 17), datetime.date(2026, 3, 16),
       datetime.date(2026, 4, 14), datetime.date(2026, 5, 14),
       datetime.date(2026, 6, 15), datetime.date(2026, 7, 14),
       datetime.date(2026, 8, 14), datetime.date(2026, 9, 14),
       datetime.date(2026, 10, 14), datetime.date(2026, 11, 16),
       datetime.date(2026, 12, 14), datetime.date(2027, 1, 14),
       datetime.date(2027, 2, 16), datetime.date(2027, 3, 15),
       datetime.date(2027, 4, 14), datetime.dat

In [90]:
filtered_df.groupby('CompID').agg({'Notional':'sum'})

,Notional
CompID,
EFA,"1,899,976,806.05"
EFC,"465,857,730.57"
INT,"287,733,064.80"


In [94]:
print(f'{filtered_df['Notional'].sum():,.0f}')

2,653,567,601


In [89]:
# Define pivot columns and rows 

# piv_rows = ['CompID', 'Series', 'HedgeDate']
piv_rows = ['HedgeDate', 'CompID', 'Series']
# Create pivot table 
filtered_df.pivot_table(index=piv_rows, columns='Bbg_Idx', values='Notional', aggfunc='sum') # Display the pivot table print(pivot_table)

Bbg_Idx                                    MXEA Index  RTY Index    SPX Index
HedgeDate  CompID Series                                                     
2021-11-15 EFA    ATHENA INDEXED UL- S153         NaN        NaN 2,167,063.01
                  Indexed UL-CV S155              NaN        NaN 2,431,405.36
                  SIUL-CV S156                    NaN        NaN    11,576.60
           EFC    ATHENA INDEXED UL- S153         NaN        NaN     5,769.52
                  Indexed UL-CV S155              NaN        NaN   201,990.65
...                                               ...        ...          ...
2026-08-15 EFC    INDEXED GROW S159               NaN        NaN   134,032.23
2026-09-15 EFA    INDEXED GROW S159               NaN        NaN   318,382.42
           EFC    INDEXED GROW S159               NaN        NaN    99,608.89
2026-10-15 EFA    INDEXED GROW S159               NaN        NaN   238,107.67
           EFC    INDEXED GROW S159               NaN        NaN    55,116.17

[421 rows x 3 columns]

In [74]:
filtered_df

,ValDt,Opt_Type,CompID,Series,FUND,Bbg_Idx,Notional,ExpiryDt,Mat(Yrs),Cap,Buffer,Part
0,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL- S151,WUF,SPX Index,"5,715.27",2027-06-14,3,0.28,1,1.00
1,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPA,SPX Index,"3,684,561.69",2025-01-14,1,0.07,1,1.00
2,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPB,SPX Index,"2,356,522.19",2025-02-14,1,0.07,1,1.00
3,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPC,SPX Index,"2,954,258.27",2025-03-14,1,0.07,1,1.00
4,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPD,SPX Index,"3,703,065.56",2025-04-14,1,0.07,1,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUI,SPX Index,"2,782,326.75",2027-09-14,3,0.28,1,1.00
1200,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUJ,SPX Index,0.16,2024-10-14,3,0.30,1,1.00
1201,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUJ,SPX Index,"1,837,676.03",2027-10-14,3,0.28,1,1.00
1202,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUK,SPX Index,"2,244,646.11",2024-11-14,3,0.30,1,1.00


In [68]:
filtered_df.columns

Index(['CompID', 'Series', 'Index_Orig', 'FUND', 'MATURITY', 'CAPR',
       'SEG_VALUE', 'SEG_PRIN_AMT', 'PROJ_CRED', 'LOAN', 'PW', 'MD',
       'Notional', 'Cap', 'Opt_Type', 'Buffer', 'ValDt', 'Bbg_Idx', 'Mat(Yrs)',
       'Part'],
      dtype='object')

In [ ]:
# Apply the function to create a new 'adjusted_date' column df['adjusted_date'] = df.apply(adjust_date, axis=1)


# filtered_df['HedgeDate'] = (filtered_df['HedgeDate'] + pd.DateOffset(years=-filtered_df['Mat(Yrs)'])).replace()

In [77]:
filtered_df

,ValDt,Opt_Type,CompID,Series,FUND,Bbg_Idx,Notional,ExpiryDt,Mat(Yrs),Cap,Buffer,Part,HedgeDate
0,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL- S151,WUF,SPX Index,"5,715.27",2027-06-14,3,0.28,1,1.00,2024-06-15
1,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPA,SPX Index,"3,684,561.69",2025-01-14,1,0.07,1,1.00,2024-01-15
2,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPB,SPX Index,"2,356,522.19",2025-02-14,1,0.07,1,1.00,2024-02-15
3,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPC,SPX Index,"2,954,258.27",2025-03-14,1,0.07,1,1.00,2024-03-15
4,2024-10-22,Call Spread,EFC,ATHENA INDEXED UL-S151 NY,WPD,SPX Index,"3,703,065.56",2025-04-14,1,0.07,1,1.00,2024-04-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUI,SPX Index,"2,782,326.75",2027-09-14,3,0.28,1,1.00,2024-09-15
1200,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUJ,SPX Index,0.16,2024-10-14,3,0.30,1,1.00,2021-10-15
1201,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUJ,SPX Index,"1,837,676.03",2027-10-14,3,0.28,1,1.00,2024-10-15
1202,2024-10-22,Call Spread,INT,ATHENA INDEXED UL- S151,WUK,SPX Index,"2,244,646.11",2024-11-14,3,0.30,1,1.00,2021-11-15


In [28]:
iul_rpt_df

,CompID,Series,Index_Orig,FUND,MATURITY,CAPR,SEG_VALUE,SEG_PRIN_AMT,PROJ_CRED,LOAN,PW,MD
0,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 1 YEAR TERM ***,WP0,,000.00,0.00,0.00,0.00,0.00,0.00,0.00
1,EFC,ATHENA INDEXED UL- S151,*** EAFE ***,WQ0,,000.00,0.00,0.00,0.00,0.00,0.00,0.00
2,EFC,ATHENA INDEXED UL- S151,*** RUSSELL ***,WR0,,000.00,0.00,0.00,0.00,0.00,0.00,0.00
3,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUA,01/14/2027,025.00,0.00,2374.40,71.06,0.00,0.00,0.00
4,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUF,06/14/2027,027.50,5715.27,355.15,0.00,0.00,0.00,837.88
...,...,...,...,...,...,...,...,...,...,...,...,...
1457,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUJ,10/14/2024,030.00,0.16,27.76,12.74,0.00,0.00,0.00
1458,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUJ,10/14/2027,027.50,1837676.03,1838818.61,1629515.98,0.00,0.00,749.16
1459,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUK,11/14/2024,030.00,2244646.11,2993500.83,2482240.81,0.00,0.00,240.33
1460,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUL,12/16/2024,030.00,895708.40,1454683.38,1021315.25,0.00,0.00,140.26


In [61]:
filtered_df

,CompID,Series,Index_Orig,FUND,MATURITY,CAPR,SEG_VALUE,SEG_PRIN_AMT,PROJ_CRED,LOAN,PW,MD
4,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUF,2027-06-14,27.50,"5,715.27",355.15,0.00,0.00,0.00,837.88
6,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPA,2025-01-14,6.50,"3,684,561.69","3,931,853.43","3,743,412.94",0.00,0.00,"4,027.65"
7,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPB,2025-02-14,6.50,"2,356,522.19","2,462,959.86","2,377,430.54",0.00,0.00,556.96
8,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPC,2025-03-14,6.50,"2,954,258.27","3,071,856.63","2,964,762.41",0.00,0.00,"1,909.97"
9,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPD,2025-04-14,6.50,"3,703,065.56","3,867,497.75","3,696,773.38",0.00,0.00,"4,326.85"
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUI,2027-09-14,27.50,"2,782,326.75","2,793,365.19","2,632,294.01",0.00,0.00,821.37
1457,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUJ,2024-10-14,30.00,0.16,27.76,12.74,0.00,0.00,0.00
1458,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUJ,2027-10-14,27.50,"1,837,676.03","1,838,818.61","1,629,515.98",0.00,0.00,749.16
1459,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUK,2024-11-14,30.00,"2,244,646.11","2,993,500.83","2,482,240.81",0.00,0.00,240.33


In [34]:
# filtered_df = iul_rpt_df[iul_rpt_df['MATURITY'].isna()]
iul_rpt_df[iul_rpt_df['MATURITY'].isna()]


,CompID,Series,Index_Orig,FUND,MATURITY,CAPR,SEG_VALUE,SEG_PRIN_AMT,PROJ_CRED,LOAN,PW,MD
0,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 1 YEAR TERM ***,WP0,NaT,000.00,0.00,0.00,0.00,0.00,0.00,0.00
1,EFC,ATHENA INDEXED UL- S151,*** EAFE ***,WQ0,NaT,000.00,0.00,0.00,0.00,0.00,0.00,0.00
2,EFC,ATHENA INDEXED UL- S151,*** RUSSELL ***,WR0,NaT,000.00,0.00,0.00,0.00,0.00,0.00,0.00
5,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WS0,NaT,000.00,0.00,0.00,0.00,0.00,0.00,0.00
34,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WP0,NaT,000.00,51745.38,0.00,0.00,-53.75,0.00,2203.70
...,...,...,...,...,...,...,...,...,...,...,...,...
1377,EFA,SIUL-CV S156,*** CH S&P 500 - 3 YEAR TERM ***,WY0,NaT,000.00,0.00,0.00,0.00,0.00,0.00,0.00
1392,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 1 YEAR TERM ***,WP0,NaT,000.00,160053.27,0.00,0.00,-142.30,0.00,3924.10
1407,INT,ATHENA INDEXED UL- S151,*** EAFE ***,WQ0,NaT,000.00,11938.00,0.00,0.00,0.00,0.00,520.87
1422,INT,ATHENA INDEXED UL- S151,*** RUSSELL ***,WR0,NaT,000.00,12056.18,0.00,0.00,-142.00,0.00,807.16


In [102]:
-1 % 7

6

In [59]:
filtered_df

,CompID,Series,Index_Orig,FUND,MATURITY,CAPR,SEG_VALUE,SEG_PRIN_AMT,PROJ_CRED,LOAN,PW,MD
4,EFC,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUF,2027-06-14,27.50,"5,715.27",355.15,0.00,0.00,0.00,837.88
6,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPA,2025-01-14,6.50,"3,684,561.69","3,931,853.43","3,743,412.94",0.00,0.00,"4,027.65"
7,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPB,2025-02-14,6.50,"2,356,522.19","2,462,959.86","2,377,430.54",0.00,0.00,556.96
8,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPC,2025-03-14,6.50,"2,954,258.27","3,071,856.63","2,964,762.41",0.00,0.00,"1,909.97"
9,EFC,ATHENA INDEXED UL-S151 NY,*** S&P 500 - 1 YEAR TERM ***,WPD,2025-04-14,6.50,"3,703,065.56","3,867,497.75","3,696,773.38",0.00,0.00,"4,326.85"
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUI,2027-09-14,27.50,"2,782,326.75","2,793,365.19","2,632,294.01",0.00,0.00,821.37
1457,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUJ,2024-10-14,30.00,0.16,27.76,12.74,0.00,0.00,0.00
1458,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUJ,2027-10-14,27.50,"1,837,676.03","1,838,818.61","1,629,515.98",0.00,0.00,749.16
1459,INT,ATHENA INDEXED UL- S151,*** S&P 500 - 3 YEAR TERM ***,WUK,2024-11-14,30.00,"2,244,646.11","2,993,500.83","2,482,240.81",0.00,0.00,240.33


In [56]:
filtered_df.describe()

,CAPR,SEG_VALUE,SEG_PRIN_AMT,PROJ_CRED,LOAN,PW,MD
count,"1,208.00","1,208.00","1,208.00","1,208.00","1,208.00","1,208.00","1,208.00"
mean,76.06,"2,196,947.24","2,286,783.35","2,206,490.04",42.15,5.18,"1,017.22"
std,239.69,"5,748,608.08","5,915,105.31","5,760,179.47",332.00,58.30,"3,148.58"
min,5.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,6.25,"17,019.30","18,373.96","17,406.40",0.00,0.00,0.00
50%,7.75,"253,133.57","265,658.40","248,673.84",0.00,0.00,50.56
75%,25.00,"1,325,973.90","1,474,587.95","1,356,943.11",0.00,0.00,453.90
max,999.99,"56,488,022.65","58,242,103.08","56,921,527.96","9,137.46","1,398.44","53,443.90"


In [42]:
filtered_df.CAPR.unique()

array(['025.00', '027.50', '006.50', '006.75', '', '007.00', '007.25',
       '007.50', '007.75', '030.00', '032.50', '999.99', '010.00',
       '010.25', '008.75', '005.75', '006.00', '006.25', '005.00',
       '005.25', '020.00', '022.50', '011.00', '005.50', '008.00'],
      dtype=object)

ValueError: could not convert string to float: ''

In [38]:
# filtered_df[filtered_df['SEG_VALUE']==0]
# filtered_df[filtered_df['CAPR']==0]
filtered_df['SEG_VALUE'].unique()

array(['0.00', '5715.27', '3684561.69', ..., '1837676.03', '2244646.11',
       '895708.40'], dtype=object)

In [20]:
for i in indices:
    print(i)

*** S&P 500 - 1 YEAR TERM ***
*** EAFE ***
*** RUSSELL ***
*** S&P 500 - 3 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** EAFE ***
*** RUSSELL ***
*** S&P 500 - 1 YEAR TERM ***
*** EAFE ***
*** RUSSELL ***
*** S&P 500 - 3 YEAR TERM ***
*** CH S&P 500 - 1 YEAR TERM ***
*** CH EAFE ***
*** CH RUSSELL ***
*** CH S&P 500 - 3 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** EAFE ***
*** RUSSELL ***
*** CH S&P 500 - 1 YEAR TERM ***
*** CH EAFE ***
*** CH RUSSELL ***
*** S&P 500 - 1 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** EAFE ***
*** RUSSELL ***
*** S&P 500 - 3 YEAR TERM ***
***PLUS S&P 500 - 1 YEAR TERM***
*** PLUS EAFE ***
*** PLUS RUSSELL ***
*** PLUS S&P 500 - 3 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** S&P 500 - 1 YEAR TERM ***
*** S&P 500 - HI PAR 1  *****
*** S&P 500 - HI PAR 2  *****
*** S&P 500 - 1 YEAR TERM ***
*** EAFE ***
*** RUSSELL ***
*** S&P 500 - 3 YEAR TERM ***
*** CH S&P 500 - 1 YEAR TERM ***
*** CH 

In [21]:
indices[0] == '*** S&P 500 - 1 YEAR TERM ***'

True

In [22]:
for l in filtered_lines:
    print(l)

*** S&P 500 - 1 YEAR TERM ***
['EFC', 'ATHENA INDEXED UL- S151', 'FUND', 'MATURITY', 'CAPR', 'SEG_VALUE', 'SEG_PRIN_AMT', 'PROJ_CRED', 'LOAN', 'PW', 'MD']
['EFC', 'ATHENA INDEXED UL- S151', 'WP0', '', '000.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
*** EAFE ***
['EFC', 'ATHENA INDEXED UL- S151', 'FUND', 'MATURITY', 'CAPR', 'SEG_VALUE', 'SEG_PRIN_AMT', 'PROJ_CRED', 'LOAN', 'PW', 'MD']
['EFC', 'ATHENA INDEXED UL- S151', 'WQ0', '', '000.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
*** RUSSELL ***
['EFC', 'ATHENA INDEXED UL- S151', 'FUND', 'MATURITY', 'CAPR', 'SEG_VALUE', 'SEG_PRIN_AMT', 'PROJ_CRED', 'LOAN', 'PW', 'MD']
['EFC', 'ATHENA INDEXED UL- S151', 'WR0', '', '000.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
*** S&P 500 - 3 YEAR TERM ***
['EFC', 'ATHENA INDEXED UL- S151', 'FUND', 'MATURITY', 'CAPR', 'SEG_VALUE', 'SEG_PRIN_AMT', 'PROJ_CRED', 'LOAN', 'PW', 'MD']
['EFC', 'ATHENA INDEXED UL- S151', 'WUA', '01/14/2027', '025.00', '0.00', '2374.40', '71.06', '0.00